# H3를 이용한 Point In Polygon

<br>

<div class="toc"><ul class="toc-item"><li><span><a href="#H3를-이용한-Point-In-Polygon" data-toc-modified-id="H3를-이용한-Point-In-Polygon-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>H3를 이용한 Point In Polygon</a></span><ul class="toc-item"><li><span><a href="#Area-(지역,-Polygon)별-H3-셀을-생성하여-셀과-AreaId를-매핑시킨다." data-toc-modified-id="Area-(지역,-Polygon)별-H3-셀을-생성하여-셀과-AreaId를-매핑시킨다.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Area (지역, Polygon)별 H3 셀을 생성하여 셀과 AreaId를 매핑시킨다.</a></span></li><li><span><a href="#사용자가-위경도를-입력하면-해당-위치의-지정한-최소-Resolution과-최대-Resolution의-H3-셀로-변환한다." data-toc-modified-id="사용자가-위경도를-입력하면-해당-위치의-지정한-최소-Resolution과-최대-Resolution의-H3-셀로-변환한다.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>사용자가 위경도를 입력하면 해당 위치의 지정한 최소 Resolution과 최대 Resolution의 H3 셀로 변환한다.</a></span></li><li><span><a href="#사용자-위경도를-변환한-List을-순회하며-해당하는-AreaId가-존재하는지-확인한다." data-toc-modified-id="사용자-위경도를-변환한-List을-순회하며-해당하는-AreaId가-존재하는지-확인한다.-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>사용자 위경도를 변환한 List을 순회하며 해당하는 AreaId가 존재하는지 확인한다.</a></span></li></ul></li></ul></div>

<br>

H3를 이용하여 Point In Polygon을 간단히 구현해보는 학습테스트입니다.

PIP를 구현하는 과정은 아래와 같다.

1. Area (지역, Polygon)별 H3 셀을 생성하여 셀과 AreaId를 매핑시킨다.
    * 최소 Resolution과 최대 Resolution을 정하여 H3셀로 최적화한다. (compact)
    * 저장형태 - `key : value` = `H3 : AreaId`
2. 사용자가 위경도를 입력하면 해당 위치의 지정한 최소 Resolution과 최대 Resolution의 H3 셀로 변환한다.
    * 위경도 -> `List<H3Cell>`
3. 사용자 위경도를 변환한 `List<H3Cell>`을 순회하며 해당하는 AreaId가 존재하는지 확인한다.
    * `H3 : AreaId`의 Key(H3)를 이용하여 AreaId를 탐색하는 것. (시간 복잡도 - O(1))

In [2]:
import folium
import h3
import geopandas as gpd
from shapely.geometry import Polygon
h3.versions()

{'c': '3.7.1', 'python': '3.7.4'}

## Area (지역, Polygon)별 H3 셀을 생성하여 셀과 AreaId를 매핑시킨다.
* 최소 Resolution과 최대 Resolution을 정하여 H3셀로 최적화한다. (compact)
* 저장형태 - `key : value` = `H3 : AreaId`

In [4]:
# 옥수동 GeoJson
oksu_area = {"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[127.01445372035877,37.54809716556332],[127.01519057405264,37.547067018782066],[127.01682578510314,37.545734088046544],[127.01971158208781,37.54514917644688],[127.02129552047927,37.543356952041016],[127.02268789647377,37.5416894537722],[127.02287476642073,37.54145247733586],[127.02502596652482,37.53872004668483],[127.0256997843901,37.53786411011561],[127.02156491849887,37.535530965877896],[127.02092441234515,37.53511935680614],[127.01719422667055,37.53742400398111],[127.01576373980122,37.5383059781119],[127.01478926311766,37.538901377441235],[127.012889891509,37.539313813299806],[127.01213660754814,37.539185649095785],[127.0121256959346,37.53915070095481],[127.01176468898241,37.53904214089355],[127.01153303960568,37.53898146832435],[127.00992598136419,37.5393714047437],[127.00947114947282,37.53957682723548],[127.00932275981805,37.53967748124033],[127.00833696037861,37.54185929062551],[127.00842516184942,37.54286466482798],[127.00857717612135,37.543889052496155],[127.00860603570271,37.5439797142403],[127.01068194070888,37.548078215345136],[127.01078235714807,37.548086518251225],[127.01081324310977,37.54809814744944],[127.01086705353879,37.54811841641107],[127.01147996885422,37.54841614618272],[127.01165764754505,37.548511522986345],[127.01320761955266,37.54994422861261],[127.01437412125692,37.54913421189207],[127.01445372035877,37.54809716556332]]]}}

In [9]:
# Area A에 대한 H3 정보를 생성한다.
min_h3_resolution = 6
base_h3_resolution = 12
area_a_h3_12 = list(h3.polyfill(oksu_area['geometry'], 12, geo_json_conformant=True))
area_a_h3_compacted = list(h3.compact(area_a_h3_12))

# 생성한 H3 정보와 Area A를 매핑한다.
db = {h3 : 'Area A' for h3 in area_a_h3_compacted}
display(db)

{'8b30e1d8b6d1fff': 'Area A',
 '8c30e1ca59a91ff': 'Area A',
 '8c30e1d8b0667ff': 'Area A',
 '8b30e1d8b15bfff': 'Area A',
 '8b30e1d8b6a9fff': 'Area A',
 '8b30e1d8b8cbfff': 'Area A',
 '8c30e1d8b1367ff': 'Area A',
 '8c30e1d8b2999ff': 'Area A',
 '8c30e1ca59319ff': 'Area A',
 '8b30e1ca5995fff': 'Area A',
 '8b30e1d8b448fff': 'Area A',
 '8b30e1d8b29efff': 'Area A',
 '8a30e1d8b117fff': 'Area A',
 '8c30e1d8b3ab1ff': 'Area A',
 '8b30e1d8b2a2fff': 'Area A',
 '8c30e1d8b44e3ff': 'Area A',
 '8c30e1d8bcc2bff': 'Area A',
 '8c30e1d8bcc01ff': 'Area A',
 '8c30e1d8b16adff': 'Area A',
 '8c30e1ca598bdff': 'Area A',
 '8c30e1d8b52dbff': 'Area A',
 '8c30e1ca59317ff': 'Area A',
 '8c30e1d8b125bff': 'Area A',
 '8c30e1ca59897ff': 'Area A',
 '8c30e1d8bcc07ff': 'Area A',
 '8c30e1ca5992dff': 'Area A',
 '8b30e1d8b68dfff': 'Area A',
 '8c30e1ca5d2c9ff': 'Area A',
 '8b30e1d8b15dfff': 'Area A',
 '8c30e1d8b3ab9ff': 'Area A',
 '8c30e1d8bc4ebff': 'Area A',
 '8a30e1d8b12ffff': 'Area A',
 '8c30e1d8b38b9ff': 'Area A',
 '8b30e1d8

<br>

## 사용자가 위경도를 입력하면 해당 위치의 지정한 최소 Resolution과 최대 Resolution의 H3 셀로 변환한다.
* 위경도 -> `List<H3Cell>`

In [15]:
# 사용자 위치정보 -> List<H3Cell> 변환 함수 정의
def geo_to_h3(lat, lng, min_resolution, base_resolution):
    result = []
    for resolution in range(min_resolution, base_resolution):
        h3_cell = h3.geo_to_h3(lat, lng, resolution)
        result.append(h3_cell)
    return result

# 사용자 위치정보 -> List<H3Cell> 변환
user_a_geo = [37.5404, 127.0184]
user_a_h3_cells = geo_to_h3(user_a_geo[0], user_a_geo[1], min_h3_resolution, base_h3_resolution)
print('user_a 위경도를 H3로 변환한 결과: ', user_a_h3_cells)

user_b_geo = [37.5448, 127.0247]
user_b_h3_cells = geo_to_h3(user_b_geo[0], user_b_geo[1], min_h3_resolution, base_h3_resolution)
print('user_a 위경도를 H3로 변환한 결과: ', user_b_h3_cells)

user_a 위경도를 H3로 변환한 결과:  ['8630e1d8fffffff', '8730e1d8bffffff', '8830e1d8b7fffff', '8930e1d8b77ffff', '8a30e1d8b757fff', '8b30e1d8b750fff']
user_a 위경도를 H3로 변환한 결과:  ['8630e1d8fffffff', '8730e1d8bffffff', '8830e1d8b3fffff', '8930e1d8b2fffff', '8a30e1d8b2effff', '8b30e1d8b2edfff']


<br>

## 사용자 위경도를 변환한 List<H3Cell>을 순회하며 해당하는 AreaId가 존재하는지 확인한다.
* `H3 : AreaId`의 `Key`(H3)를 이용하여 AreaId를 탐색하는 것. (시간 복잡도 - O(1))

In [21]:
# 유저가 속하는 Area 반환 함수 정의
def what_area_is_in(db, h3_cells):
    result = set()
    for cell in h3_cells:
        if (db.get(cell) is not None):
            result.add(db.get(cell))
    return result

# 유저가 속하는 Area 확인
user_a_result = what_area_is_in(db, user_a_h3_cells)
user_b_result = what_area_is_in(db, user_b_h3_cells)

# 결과 출력
print(f'유저 a가 속하는 지점은 {"없습니다" if not user_a_result else user_a_result}')
print(f'유저 b가 속하는 지점은 {"없습니다" if not user_b_result else user_b_result}')

유저 a가 속하는 지점은 {'Area A'}
유저 b가 속하는 지점은 없습니다
